In [1]:
import torchvision
import torch
import importlib
import resnet
import numpy as np
from torchvision import transforms
import cifar10_resized
import pickle
import json
import utils.utils as utils
from utils.accumulator import Accumulator
import train
importlib.reload(train)
importlib.reload(cifar10_resized)

<module 'cifar10_resized' from '/juice/scr/ananya/cifar_experiments/custom_training/cifar10_resized.py'>

In [2]:
# Read config.
# Load checkpoint
config_path = './logs/small_run_merlin_7/config.json'
checkpoint_path = './logs/small_run_merlin_7/checkpoints/ckp_best'
batch_size, num_workers = 128, 16
use_cuda = True
with open(config_path) as f:
    device = 'cuda'
    config = json.load(f)
net = utils.initialize(config['model'])
if use_cuda:
    net.cuda()
utils.load_ckp(checkpoint_path, net)
test_data = utils.init_dataset(config['test_dataset'])
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=batch_size,
    shuffle=False, num_workers=num_workers)
criterion = utils.initialize(config['criterion'])

In [3]:
val_loss, val_acc = train.get_test_stats(config, net, test_loader, criterion, device)

In [4]:
val_acc.get_mean()

0.3026

# Tuning the last linear layer which doesn't really work

In [5]:
# Train linear model on top of features.
net.eval()
features_list, logits_list, labels_list = [], [], []
with torch.no_grad():
    for data in test_loader:
        if use_cuda:
            data = utils.to_device(data, device)
        images, labels = data
        outputs, features = net(images, return_features_also=True)
        logits_list.append(outputs.detach().cpu().numpy())
        features_list.append(features.detach().cpu().numpy())
        labels_list.append(labels.detach().cpu().numpy())
features = np.concatenate(features_list)
labels = np.concatenate(labels_list)
logits = np.concatenate(logits_list)
print(features.shape, labels.shape, logits.shape)

(10000, 512) (10000,) (10000, 10)


In [6]:
# Select tuning and testing indices
seed = 0
num_tune = 5000
num_test = len(test_data) - num_tune
rng = np.random.default_rng(seed=seed)
indices = rng.permutation(len(test_data))
tune_indices = indices[:num_tune]
test_indices = indices[num_tune:]

In [7]:
# Get model accuracy on second part
test_logits = logits[test_indices]
test_preds = np.argmax(test_logits, axis=1)
test_labels = labels[test_indices]
print(np.mean(test_preds==test_labels))

0.3006


In [14]:
# Train sklearn linear probe on first part, measure acc on second part
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
tune_features = features[tune_indices]
test_features = features[test_indices]
tune_labels = labels[tune_indices]
test_labels = labels[test_indices]
# clf = RandomForestClassifier(max_depth=3, random_state=0).fit(tune_features, tune_labels)
clf = LogisticRegression(random_state=0, C=0.1).fit(tune_features, tune_labels)

/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [15]:
# Get training acc
from scipy.special import softmax
tune_preds = clf.predict(tune_features)
tune_acc = np.mean(tune_preds==tune_labels)
print(tune_acc)
test_preds = clf.predict(test_features)
test_pred_probs = clf.predict_proba(test_features)
test_acc = np.mean(test_preds==test_labels)
print(test_acc)

0.3084
0.3102


In [34]:
test_probs = softmax(test_logits, axis=1)
print(np.mean(np.max(test_probs, axis=1)))

0.9394416


In [35]:
print(np.mean(np.max(test_pred_probs, axis=1)))

0.812295134468732
